In [14]:
import pandas as pd 
import yaml

def load_yaml(load_path):
    responses_file = open(load_path, 'r')
    responses = responses_file.read()
    responses = yaml.safe_load(responses)
    return responses

df_raw = load_yaml("reasons2.yml")
df_raw['reasons'][:3]

[{'reason': 'control_weight',
  'label': 0,
  'examples': "i am overweighted, so i want to lose weight.\ni want to control the weight.\ni don't want to be overweighted.\nexercise can enhance my body and muscle.\nI want fitness.\ni want more muscle.\ni want better body.\nkeep fit\n"},
 {'reason': 'against_diseases',
  'label': 1,
  'examples': 'exercise can help me overcome the sickness.\nexercise can help me better recover.\nagainst the disease.\n'},
 {'reason': 'decrease_risk_heart_attack',
  'label': 2,
  'examples': 'suitable activity can decrease the risk of heart attack.\ni do not want to have risk of heart attack.\nreduse rish of heart attack.\ngood for my heart.\n'}]

In [15]:
def data_reformulate(df_raw):
    reason, text, label = [], [], []
    for i in df_raw['reasons']:
        example = i['examples'].strip().split('\n')
        for rea in example:
            reason.append(i['reason'])
            text.append(rea)
            label.append(i['label'])
    df_reason = pd.DataFrame({'reason':reason, 'label':label, 'text':text})
    return df_reason

df_reason = data_reformulate(df_raw)

In [16]:
df_reason#.head()

,reason,label,text
0,control_weight,0,"i am overweighted, so i want to lose weight."
1,control_weight,0,i want to control the weight.
2,control_weight,0,i don't want to be overweighted.
3,control_weight,0,exercise can enhance my body and muscle.
4,control_weight,0,I want fitness.
5,control_weight,0,i want more muscle.
6,control_weight,0,i want better body.
7,control_weight,0,keep fit
8,against_diseases,1,exercise can help me overcome the sickness.
9,against_diseases,1,exercise can help me better recover.


In [6]:
list(df_reason['reason'].values)[:3]

['control_weight', 'control_weight', 'control_weight']

In [17]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_reason)

In [18]:
dataset #['reason']

Dataset({
    features: ['reason', 'label', 'text'],
    num_rows: 22
})

In [19]:
dataset = dataset.shuffle()
train_dataset = dataset.train_test_split(test_size=0.1)['train']
test_dataset = dataset.train_test_split(test_size=0.1)['test']
test_dataset

Dataset({
    features: ['reason', 'label', 'text'],
    num_rows: 3
})

In [20]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=9)

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', max_length=28, truncation=True)
    

batch_size = 2
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=batch_size)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /Users/xinsun/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /Users/xinsun/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /Users/xinsun/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca

In [12]:
# train_dataset['input_ids']

In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: reason, text.
***** Running training *****
  Num examples = 19
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 50
100%|██████████| 50/50 [01:45<00:00,  1.28s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 50/50 [01:45<00:00,  2.11s/it]

{'train_runtime': 105.7965, 'train_samples_per_second': 0.898, 'train_steps_per_second': 0.473, 'train_loss': 1.917626190185547, 'epoch': 5.0}


TrainOutput(global_step=50, training_loss=1.917626190185547, metrics={'train_runtime': 105.7965, 'train_samples_per_second': 0.898, 'train_steps_per_second': 0.473, 'train_loss': 1.917626190185547, 'epoch': 5.0})

In [23]:
trainer.save_model('less_label_model/')

trainer.evaluate()

Saving model checkpoint to less_label_model/
Configuration saved in less_label_model/config.json
Model weights saved in less_label_model/pytorch_model.bin
tokenizer config file saved in less_label_model/tokenizer_config.json
Special tokens file saved in less_label_model/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: reason, text.
***** Running Evaluation *****
  Num examples = 3
  Batch size = 2
100%|██████████| 2/2 [00:00<00:00,  2.04it/s]


{'eval_loss': 1.6894389390945435,
 'eval_accuracy': 0.6666666666666666,
 'eval_f1': 0.6666666666666666,
 'eval_precision': 0.8333333333333334,
 'eval_recall': 0.6666666666666666,
 'eval_runtime': 1.9359,
 'eval_samples_per_second': 1.55,
 'eval_steps_per_second': 1.033,
 'epoch': 5.0}

In [14]:
# %load_ext tensorboard
# %tensorboard --logdir logs
# %tensorboard --logdir=data/ --host localhost --port 8000

### Test

In [24]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained("/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/ask_reason/less_label_model")
tokenizer = BertTokenizer.from_pretrained("/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/ask_reason/less_label_model")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

loading configuration file /Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/ask_reason/less_label_model/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
 

In [26]:
text = 'keep fit'
print(f'text: {text}, res: {classifier(text)}\n')

text = 'I want to sleep better'
print(f'text: {text}, res: {classifier(text)}\n')

text = 'reduce stress'
print(f'text: {text}, res: {classifier(text)}\n')

text: keep fit, res: [{'label': 'LABEL_0', 'score': 0.1666778326034546}]

text: I want to sleep better, res: [{'label': 'LABEL_4', 'score': 0.19574789702892303}]

text: reduce stress, res: [{'label': 'LABEL_3', 'score': 0.2150663584470749}]



---

### hyperparameter_search

In [15]:
def model_init():
    return BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=9)

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    # learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs'
)

In [20]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/xinsun/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-

{'train_runtime': 68.5662, 'train_samples_per_second': 1.896, 'train_steps_per_second': 0.073, 'train_loss': 2.2203540802001953, 'epoch': 5.0}


100%|██████████| 2/2 [00:00<00:00,  7.11it/s]/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 2/2 [00:00<00:00,  5.26it/s]
[I 2022-01-06 20:12:31,479] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 1.022970997029937e-06, 'num_train_epochs': 5, 'seed': 14, 'per_device_train_batch_size': 64}. Best is trial 0 with value: 0.0.
Trial:
loading config

{'train_runtime': 66.3303, 'train_samples_per_second': 1.96, 'train_steps_per_second': 0.075, 'train_loss': 2.177585220336914, 'epoch': 5.0}


100%|██████████| 2/2 [00:00<00:00,  7.88it/s]/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 2/2 [00:00<00:00,  7.14it/s]
[I 2022-01-06 20:13:44,452] Trial 1 finished with value: 0.3503401360544217 and parameters: {'learning_rate': 2.085429275130855e-05, 'num_train_epochs': 5, 'seed': 8, 'per_device_train_batch_size': 64}. Best is trial 1 with value: 0.3503401360

...

100%|██████████| 2/2 [00:00<00:00,  2.96it/s]/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 2/2 [00:00<00:00,  2.76it/s]
[I 2022-01-06 20:19:08,547] Trial 7 finished with value: 0.0 and parameters: {'learning_rate': 1.3470812037842516e-06, 'num_train_epochs': 2, 'seed': 31, 'per_device_train_batch_size': 8}. Best is trial 4 with value: 0.7800453514739228.
Trial:

{'train_runtime': 94.8952, 'train_samples_per_second': 1.096, 'train_steps_per_second': 0.169, 'train_loss': 2.2379071712493896, 'epoch': 4.0}


100%|██████████| 2/2 [00:00<00:00,  6.35it/s]/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 2/2 [00:00<00:00,  5.92it/s]
[I 2022-01-06 20:20:50,419] Trial 8 finished with value: 0.0 and parameters: {'learning_rate': 2.1454115931911646e-06, 'num_train_epochs': 4, 'seed': 39, 'per_device_train_batch_size': 8}. Best is trial 4 with value: 0.7800453514739228.
Trial:

{'train_runtime': 61.3741, 'train_samples_per_second': 1.271, 'train_steps_per_second': 0.342, 'train_loss': 2.1152926853724887, 'epoch': 3.0}


100%|██████████| 2/2 [00:00<00:00,  7.38it/s]/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 2/2 [00:00<00:00,  6.95it/s]
[I 2022-01-06 20:21:57,573] Trial 9 finished with value: 0.3503401360544217 and parameters: {'learning_rate': 1.8939015595442943e-05, 'num_train_epochs': 3, 'seed': 12, 'per_device_train_batch_size': 4}. Best is trial 4 with value: 0.780045351

In [21]:
best_run

BestRun(run_id='4', objective=0.7800453514739228, hyperparameters={'learning_rate': 1.4325496267384845e-06, 'num_train_epochs': 2, 'seed': 28, 'per_device_train_batch_size': 4})

In [22]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)
    # print(f'n: {n}, v: {v}\n')

In [271]:
# trainer.args

In [23]:
trainer.train()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/xinsun/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-

{'train_runtime': 45.762, 'train_samples_per_second': 1.136, 'train_steps_per_second': 0.306, 'train_loss': 2.2256556919642856, 'epoch': 2.0}


TrainOutput(global_step=14, training_loss=2.2256556919642856, metrics={'train_runtime': 45.762, 'train_samples_per_second': 1.136, 'train_steps_per_second': 0.306, 'train_loss': 2.2256556919642856, 'epoch': 2.0})

In [24]:
trainer.save_model('hypersearch_model/')

trainer.evaluate()

Saving model checkpoint to hypersearch_model/
Configuration saved in hypersearch_model/config.json
Model weights saved in hypersearch_model/pytorch_model.bin
tokenizer config file saved in hypersearch_model/tokenizer_config.json
Special tokens file saved in hypersearch_model/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, reason.
***** Running Evaluation *****
  Num examples = 7
  Batch size = 4
100%|██████████| 2/2 [00:00<00:00,  6.51it/s]/Users/xinsun/anaconda3/envs/chatbot/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 2/2 [00:00<00:00,  5.38it/s]


{'eval_loss': 2.176393747329712,
 'eval_accuracy': 0.2857142857142857,
 'eval_f1': 0.126984126984127,
 'eval_precision': 0.08163265306122448,
 'eval_recall': 0.2857142857142857,
 'eval_runtime': 0.8489,
 'eval_samples_per_second': 8.246,
 'eval_steps_per_second': 2.356,
 'epoch': 2.0}

In [26]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained("/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/ask_reason/hypersearch_model")
tokenizer = BertTokenizer.from_pretrained("/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/ask_reason/hypersearch_model")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

loading configuration file /Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/ask_reason/hypersearch_model/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,


In [27]:
text = 'keep fit'
print(f'text: {text}, res: {classifier(text)}\n')

text = 'i can have better sleep'
print(f'text: {text}, res: {classifier(text)}\n')

text = 'reduce stress'
print(f'text: {text}, res: {classifier(text)}\n')

text: keep fit, res: [{'label': 'LABEL_0', 'score': 0.16638430953025818}]

text: i can have better sleep, res: [{'label': 'LABEL_0', 'score': 0.18501438200473785}]

text: reduce stress, res: [{'label': 'LABEL_0', 'score': 0.17209725081920624}]



In [28]:
text = 'reduce the risk of heart attack'
print(f'text: {text}, res: {classifier(text)}\n')

text: reduce the risk of heart attack, res: [{'label': 'LABEL_0', 'score': 0.17742426693439484}]



In [10]:
import random

a = "asd"

response_pro = [(f'If I am understand correctly, you think the pros of people doing exercise and physical activity is {a}'),
            (f'I get it, {a}')]

print(random.choice(response_pro))

If I am understand correctly, you think the pros of people doing exercise and physical activity is asd
